In [ ]:
import os
import sqlite3
import pandas as pd

In [ ]:
con = sqlite3.connect('./noahs.sqlite')
df = pd.read_sql_query('select * from customers', con)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# ['Jr.', 'II', 'III', 'IV', 'V'] are not last names
m = df['name'].str.split(' ').apply(len) != 2
thirds = df[m]['name'].str.split(' ').apply(lambda x: x[-1])
thirds.value_counts().head(10)

In [ ]:
def parse_name(inp):
    s = inp.split(' ')
    name = s[:-1] if s[-1] in ['Jr.', 'II', 'III', 'IV', 'V'] else s
    output = {
        'first': name[0],
        'middle': name[1] if len(name) > 2 else None,
        'last': name[-1],
    }
    return output

In [ ]:
jdf = df.join(pd.json_normalize(df['name'].apply(parse_name).values))

In [ ]:
alpha = 'abcdefghijklmnopqrstuvwxyz'
nums = []
n = 2
j = 0
for i, char in enumerate(alpha):
    nums.append(n)
    k = 3 if n in (7, 9) else 2
    if j == k:
        n += 1
        j = 0
    else:
        j += 1
t9 = dict(zip(alpha, nums))

In [ ]:
jdf['last_phone'] = jdf['last'].apply(lambda x: ''.join([str(t9[c]) for c in x.lower()]))
jdf['phone_nodash'] = jdf['phone'].str.replace('-', '')

In [ ]:
jdf[jdf['last_phone'] == jdf['phone_nodash']]['phone']